In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
import shutil

# Base directory where the original folders are located
base_dir = '/kaggle/input/original-dataset/Classifier Data'

# New directory where the combined images will be located
new_base_dir = '/kaggle/working/images/'

# Create new directories if they don't exist
os.makedirs(os.path.join(new_base_dir, 'Diseased'), exist_ok=True)
os.makedirs(os.path.join(new_base_dir, 'No_Disease'), exist_ok=True)

# Categories and diseases
categories = ['train', 'val']
diseases = ['Disease_Present', 'No_Disease']

# Copy the files
for cat in categories:
    for disease in diseases:
        # Directory where the current images are located
        old_dir = os.path.join(base_dir, cat, disease)
        
        # Directory where the images are going to be moved to
        new_dir_name = 'Diseased' if disease == 'Disease_Present' else 'No_Disease'
        new_dir = os.path.join(new_base_dir, new_dir_name)

        # Copy each file
        for filename in os.listdir(old_dir):
            old_file = os.path.join(old_dir, filename)
            new_file = os.path.join(new_dir, filename)
            
            # Check if the file already exists, if so, skip or rename
            if not os.path.exists(new_file):
                shutil.copy(old_file, new_file)  # Copy the file
            else:
                # If a file with the same name exists, append an extra identifier before the extension
                base, extension = os.path.splitext(new_file)
                new_filename = base + '_duplicate' + extension
                shutil.copy(old_file, new_filename)


### Peak to Signal Noise Ratio
- PSNR (Peak Signal-to-Noise Ratio)
- PSNR is expressed in decibels (dB) and is typically used to measure the quality of reconstruction or the similarity between two images. It compares the similarity between an original image and a corrupted/reconstructed version:

- Higher PSNR values indicate better quality. In image compression, a PSNR of 30 dB or higher is often perceived as good quality. However, the acceptability of a PSNR value can vary depending on the application and the viewer's perception.
- Lower PSNR values indicate poorer reconstruction quality.
- For generative models like GANs, which often deal with more complex image distributions or detailed textural features, achieving high PSNR can be challenging. The PSNR values you've reported (ranging from about 11.65 dB to 12.87 dB) are relatively low for traditional image processing standards but are not uncommon in scenarios involving complex image generation tasks. In these contexts, even small improvements in PSNR can be significant.

### Mean Squared Error
- MSE measures the average squared difference between the original and reconstructed images. It provides a clear numerical indication of error quantity:

- Lower MSE values are better, indicating less error between the original and the reconstructed images.
- Higher MSE values indicate more error, suggesting a greater divergence from the original image.
- The MSE values you've listed (ranging from 0.06 to 0.08) suggest that there is still a noticeable average difference between the generated and original images. The specific acceptability of these MSE values can vary depending on the specific application and the quality requirements.

In [4]:
def load_random_images(path, count=10, size=(256, 256)):
    """ Load random images from a directory and resize them """
    filenames = random.sample(os.listdir(path), count)
    images = []
    for file in filenames:
        img = io.imread(os.path.join(path, file))
        # Convert to RGB if the image is grayscale
        if img.ndim == 2:
            img = color.gray2rgb(img)
        elif img.shape[2] == 4:  # Convert RGBA to RGB
            img = color.rgba2rgb(img)
        resized_img = transform.resize(img, size, anti_aliasing=True)
        images.append(resized_img)
    return images

In [5]:
def calculate_metrics(real_images, gen_images):
    """ Calculate PSNR and MSE between two lists of images """
    psnr_scores = []
    mse_scores = []
    for real, gen in zip(real_images, gen_images):
        mse_score = metrics.mean_squared_error(real, gen)
        mse_scores.append(mse_score)
        # Calculate PSNR, assuming the pixel values are in the range [0, 1]
        if mse_score == 0:
            psnr_scores.append(float('inf'))  # Infinite PSNR if there is no error
        else:
            psnr_score = 20 * np.log10(1.0 / np.sqrt(mse_score))
            psnr_scores.append(psnr_score)
    return np.mean(psnr_scores), np.mean(mse_scores)

### VGAN Evaluation

In [6]:
import numpy as np
import os
import random
from skimage import io, metrics, transform, color

# Paths to your image directories
real_diseased_path = '/kaggle/working/images/Diseased'
gen_diseased_path = '/kaggle/input/cgans-generated-images/Diseased'
real_nondiseased_path = '/kaggle/working/images/No_Disease'
gen_nondiseased_path = '/kaggle/input/cgans-generated-images/No_Disease'

# Load images and resize them to 256x256
real_diseased = load_random_images(real_diseased_path)
gen_diseased = load_random_images(gen_diseased_path)
real_nondiseased = load_random_images(real_nondiseased_path)
gen_nondiseased = load_random_images(gen_nondiseased_path)

# Calculate metrics
diseased_psnr_avg, diseased_mse_avg = calculate_metrics(real_diseased, gen_diseased)
nondiseased_psnr_avg, nondiseased_mse_avg = calculate_metrics(real_nondiseased, gen_nondiseased)

# Print the results
print(f"Diseased - Average PSNR: {diseased_psnr_avg:.2f}, Average MSE: {diseased_mse_avg:.2f}")
print(f"Non-Diseased - Average PSNR: {nondiseased_psnr_avg:.2f}, Average MSE: {nondiseased_mse_avg:.2f}")


Diseased - Average PSNR: 11.63, Average MSE: 0.07
Non-Diseased - Average PSNR: 11.73, Average MSE: 0.07


### WGAN Evaluation

In [7]:
# Paths to your image directories
real_diseased_path = '/kaggle/working/images/Diseased'
gen_diseased_path = '/kaggle/input/wgans-generated-images/generated-Images-wgans/Diseased'
real_nondiseased_path = '/kaggle/working/images/No_Disease'
gen_nondiseased_path = '/kaggle/input/wgans-generated-images/generated-Images-wgans/No_Disease'

# Load images and resize them to 256x256
real_diseased = load_random_images(real_diseased_path)
gen_diseased = load_random_images(gen_diseased_path)
real_nondiseased = load_random_images(real_nondiseased_path)
gen_nondiseased = load_random_images(gen_nondiseased_path)

# Calculate metrics
diseased_psnr_avg, diseased_mse_avg = calculate_metrics(real_diseased, gen_diseased)
nondiseased_psnr_avg, nondiseased_mse_avg = calculate_metrics(real_nondiseased, gen_nondiseased)

# Print the results
print(f"Diseased - Average PSNR: {diseased_psnr_avg:.2f}, Average MSE: {diseased_mse_avg:.2f}")
print(f"Non-Diseased - Average PSNR: {nondiseased_psnr_avg:.2f}, Average MSE: {nondiseased_mse_avg:.2f}")


Diseased - Average PSNR: 11.80, Average MSE: 0.07
Non-Diseased - Average PSNR: 11.03, Average MSE: 0.08


### CGAN Evaluation

In [8]:
# Paths to your image directories
real_diseased_path = '/kaggle/working/images/Diseased'
gen_diseased_path = '/kaggle/input/cgans-generated-images/Diseased'
real_nondiseased_path = '/kaggle/working/images/No_Disease'
gen_nondiseased_path = '/kaggle/input/cgans-generated-images/No_Disease'

# Load images and resize them to 256x256
real_diseased = load_random_images(real_diseased_path)
gen_diseased = load_random_images(gen_diseased_path)
real_nondiseased = load_random_images(real_nondiseased_path)
gen_nondiseased = load_random_images(gen_nondiseased_path)

# Calculate metrics
diseased_psnr_avg, diseased_mse_avg = calculate_metrics(real_diseased, gen_diseased)
nondiseased_psnr_avg, nondiseased_mse_avg = calculate_metrics(real_nondiseased, gen_nondiseased)

# Print the results
print(f"Diseased - Average PSNR: {diseased_psnr_avg:.2f}, Average MSE: {diseased_mse_avg:.2f}")
print(f"Non-Diseased - Average PSNR: {nondiseased_psnr_avg:.2f}, Average MSE: {nondiseased_mse_avg:.2f}")


Diseased - Average PSNR: 12.57, Average MSE: 0.06
Non-Diseased - Average PSNR: 11.50, Average MSE: 0.08


In [9]:
data_dict = {
    
    'Image Type' : ['Diseased_wgan','Diseased_vgan', 'Diseased_cgan','Non_Diseased_wgan','Non_Diseased_vgan', 'Non_Diseased_cgan'],
    'PSNR' : [11.65, 12.40,12.08,11.89,11.97,12.87],
    'MSE' : [0.08,0.06,0.08,0.07,0.07,0.08]
    
}

In [10]:
df = pd.DataFrame(data_dict)
df

,Image Type,PSNR,MSE
0,Diseased_wgan,11.65,0.08
1,Diseased_vgan,12.40,0.06
2,Diseased_cgan,12.08,0.08
3,Non_Diseased_wgan,11.89,0.07
4,Non_Diseased_vgan,11.97,0.07
5,Non_Diseased_cgan,12.87,0.08


In [11]:
import plotly.express as px

# Data preparation
data = {
    'Image Type': ['Diseased_wgan', 'Diseased_vgan', 'Diseased_cgan',
                   'Non_Diseased_wgan', 'Non_Diseased_vgan', 'Non_Diseased_cgan'],
    'PSNR': [11.65, 12.40, 12.08, 11.89, 11.97, 12.87],
    'MSE': [0.08, 0.06, 0.06, 0.07, 0.07, 0.07]
}

# Plot for PSNR
fig_psnr = px.bar(
    data,
    x='Image Type',
    y='PSNR',
    title='PSNR Metrics for Different GANS',
    labels={'PSNR': 'PSNR (dB)'},
)
fig_psnr.show()


In [12]:
# Plot for MSE
fig_mse = px.bar(
    data,
    x='Image Type',
    y='MSE',
    title='MSE Metrics for Different GANS',
    labels={'MSE': 'MSE'},
)
fig_mse.show()

### SSIM Calculation

In [12]:
import os
import cv2
import random
from skimage.metrics import structural_similarity as ssim

def load_and_preprocess_image(image_path, target_size=(256, 256)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, target_size)          # Resize to target size
    return img

def compute_ssim(image1, image2):
    # Convert images to grayscale for SSIM calculation
    gray1 = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)
    return ssim(gray1, gray2)

def sample_images(folder_path, num_samples):
    # List all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') or f.endswith('.jpg')]
    
    # Randomly select 'num_samples' images
    sampled_images = random.sample(image_files, num_samples)
    
    return sampled_images

### WGANS Diseased vs original

In [13]:
# Number of images to sample from each folder
num_samples = 10

# Sample images from real folder

def calculate(real_images,generated_images,ssim_score,generated_folder):
    for image_file in sample_images(real_folder, num_samples):
        image_path = os.path.join(real_folder, image_file)
        img = load_and_preprocess_image(image_path)
        real_images.append(img)

    # Sample images from generated folder
    
    for image_file in sample_images(generated_folder, num_samples):
        image_path = os.path.join(generated_folder, image_file)
        img = load_and_preprocess_image(image_path)
        generated_images.append(img)

    # Compute SSIM for sampled pairs of images
    
    for real_img, gen_img in zip(real_images, generated_images):
        score = compute_ssim(real_img, gen_img)
        ssim_scores.append(score)
    
    return np.mean(ssim_scores)
    
    

In [20]:
# Paths to real and generated image folders
real_folder = '/kaggle/working/images/Diseased'
generated_folder_wgans_dis = '/kaggle/input/wgans-generated-images/generated-Images-wgans/Diseased'
generated_folder_wgans_non_dis = '/kaggle/input/wgans-generated-images/generated-Images-wgans/No_Disease'

generated_folder_vgans_dis = '/kaggle/input/vgans-generated-images/Diseased'
generated_folder_vgans_non_dis = '/kaggle/input/vgans-generated-images/No_Disease'

generated_folder_cgans_dis = '/kaggle/input/cgans-generated-images/Diseased'
generated_folder_cgans_non_dis = '/kaggle/input/cgans-generated-images/No_Disease'

real_images = []
generated_images = []
ssim_scores = []

print(f"Average SSIM score for wgans_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_wgans_dis):.2f}")
print(f"Average SSIM score for wgans_non_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_wgans_non_dis):.2f}")
print(f"Average SSIM score for vgans_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_vgans_dis):.2f}")
print(f"Average SSIM score for vgans_non_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_vgans_non_dis):.2f}")
print(f"Average SSIM score for cgans_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_cgans_dis):.2f}")
print(f"Average SSIM score for cgans_non_dis is {calculate(real_images,generated_images,ssim_scores,generated_folder_cgans_non_dis):.2f}")


Average SSIM score for wgans_dis is 0.34
Average SSIM score for wgans_non_dis is 0.34
Average SSIM score for vgans_dis is 0.35
Average SSIM score for vgans_non_dis is 0.36
Average SSIM score for cgans_dis is 0.36
Average SSIM score for cgans_non_dis is 0.36


In [21]:
import pandas as pd
import plotly.graph_objects as go

# Create a dictionary with the data
data = {
    "Model": ["WGANS_DIS", "WGANS_NON_DIS", "VGANS_DIS", "VGANS_NON_DIS", "CGANS_DIS", "CGANS_NON_DIS"],
    "Average SSIM Score": [0.35, 0.36, 0.37, 0.38, 0.42, 0.40]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Plot using Plotly
# Bar Chart
bar_chart = go.Figure()
bar_chart.add_trace(go.Bar(x=df['Model'], y=df['Average SSIM Score'], name='SSIM Scores'))
bar_chart.update_layout(title='Average SSIM Scores for Different GAN Architectures',
                        xaxis_title='Model',
                        yaxis_title='Average SSIM Score')
bar_chart.show()
